In [320]:
from ortools.linear_solver import pywraplp

In [321]:
solver = pywraplp.Solver("Logistics: ", pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

In [322]:
# x_11 + x_12 + x_13 <= 100
# x_21 + x_22 + x_23 <= 120
# x_11 + x_21 = 80
# x_12 + x_22 = 70
# x-13 + x_23 = 50
# x_ij >= 0
# min: 4x_11+6x_12+9x_13+5x_21+4x_22+7x_23

x_11 = solver.IntVar(0,solver.infinity(), "x_11")
x_12 = solver.IntVar(0,solver.infinity(), "x_12")
x_13 = solver.IntVar(0,solver.infinity(), "x_13")

x_21 = solver.IntVar(0,solver.infinity(), "x_21")
x_22 = solver.IntVar(0,solver.infinity(), "x_22")
x_23 = solver.IntVar(0,solver.infinity(), "x_23")


constraint1 = solver.Constraint(0,100)
constraint1.SetCoefficient(x_11,1)
constraint1.SetCoefficient(x_12,1)
constraint1.SetCoefficient(x_13,1)

constraint2 = solver.Constraint(0,120)
constraint2.SetCoefficient(x_21,1)
constraint2.SetCoefficient(x_22,1)
constraint2.SetCoefficient(x_23,1)

solver.Add(sum([x_11, x_21])== 80)
# solver.Add(x_12 + x_22== 70)
solver.Add(x_13 + x_23== 50)

constraint3 = solver.Constraint(70,70)
constraint3.SetCoefficient(x_12, 1)
constraint3.SetCoefficient(x_22, 1)


d = {"x_12": x_12, "x_22": x_22}

print(d)

obj = [d[list(d.keys())[i]] for i in range(len(d.keys()))]
import numpy as np
print(obj)
print(np.sum(obj))
# l = sum([x_12, x_22])
# solver.Add(l == 70)

{'x_12': x_12, 'x_22': x_22}
[x_12, x_22]
(x_12 + x_22)


In [323]:
objective = solver.Objective()
objective.SetCoefficient(x_11, 4)
objective.SetCoefficient(x_12, 6)
objective.SetCoefficient(x_13, 9)
objective.SetCoefficient(x_21, 5)
objective.SetCoefficient(x_22, 4)
objective.SetCoefficient(x_23, 7)
objective.SetMinimization()

In [324]:
result = solver.Solve()
print(x_11.solution_value())
print(x_12.solution_value())
print(x_13.solution_value())
print(x_21.solution_value())
print(x_22.solution_value())
print(x_23.solution_value())

80.0
0.0
0.0
0.0
70.0
50.0


In [325]:


# constraint1 = solver.Constraint(80,80)
# constraint1.SetCoefficient(x_11,1)
# constraint1.SetCoefficient(x_21,1)
# 
# constraint2 = solver.Constraint(70,70)
# constraint2.SetCoefficient(x_12,1)
# constraint2.SetCoefficient(x_22,1)
# 
# constraint3 = solver.Constraint(50,50)
# constraint3.SetCoefficient(x_13,1)
# constraint3.SetCoefficient(x_23,1)


# solver.Minimize(4*x_11+6*x_12+9*x_13+5*x_21+4*x_22+7*x_23)

In [326]:
sol = solver.Solve()

print("Cost:", solver.Objective().Value())
print("x_12:", x_11.solution_value())
print("x_13:", x_12.solution_value())
print("x_11:", x_13.solution_value())
print("x_21:", x_21.solution_value())
print("x_22:", x_22.solution_value())
print("x_23:", x_23.solution_value())

Cost: 950.0
x_12: 80.0
x_13: 0.0
x_11: 0.0
x_21: 0.0
x_22: 70.0
x_23: 50.0


In [333]:
# comparar con distinto número de almacenes, 1, 2, combinaciones de los 2, suponer que hay 1 tercer almacén
# Automatizar con una funcion que reciba las matrices de datos y devuelva el resultado
import numpy as np


def get_coeffs(c):
    vars = {}
    s= 0
    for i in range(0,len(c)):
        vars["x{0}".format(s)] = c[i]
        s += 1
    return vars


def optimize(c, A_ub, b_ub, A_eq=None, b_eq=None, minmax = 0):
    """
    Optimizes a set problem given the constraints, assuming discrete variables and every variable is equal or greater than 1.

    Args:
        c (list): Coefficients of the function to minimize
        A1 (list): Coefficients for the lhs of the inequality constraints
        b1 (list): Coefficients for the rhs of the inequality constraints
        A2 (list): Coefficients for the lhs of the equality constraints
        b2 (list): Coefficients for the rhs of the equality constraints

    Returns:
        optimum (int): Optimized cost of the function
        x (list): each value of the variables used on the problem
    """

    # Initialization of the model and the variables
    solver = pywraplp.Solver("Optimization problem", pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    if minmax == 1:
        c = [-elem for elem in c]
        

    else:
        pass

    vars = get_coeffs(c)
    intvar = list(vars.keys())
    intvars = {}
    print(intvar)
    for i in range(len(vars)):
        intvars["x{0}".format(i)] = solver.IntVar(0,solver.infinity(), "{0}".format(intvar[i]))
    print(type(intvars[list(intvars.keys())[1]]))    
    # Upper bound constraints
    constraints_ub = {}
    for i in range(len(A_ub)):
        constraints_ub["constraint{0}".format(i)] = solver.Constraint(0,b_ub[i])
        setconstraint = constraints_ub[list(constraints_ub.keys())[i]]
        for j in range(len(A_ub[i])):
            if A_ub != 0:
                setconstraint.SetCoefficient(intvars[list(intvars.keys())[j]],A_ub[i][j])
    
    # Equality constraints 
    if A_eq is None:
        pass
    else:
        constraints_eq = {}
        for i in range(len(A_eq)):
            constraints_eq["constraint{0}".format(i)] = solver.Constraint(b_eq[i],b_eq[i])
            setconstraint = constraints_eq[list(constraints_eq.keys())[i]]
            for j in range(len(A_eq[i])):
                if A_eq != 0:
                    setconstraint.SetCoefficient(intvars[list(intvars.keys())[j]],A_eq[i][j])
    
    # Set objective function coefficients
    objective = solver.Objective()
    for i in range(len(vars)):
        objective.SetCoefficient(intvars[list(intvars.keys())[i]], c[i])
    
    # Show results
    print("Optimization completed")

    result = solver.Solve()
    sol = solver.Objective().Value()
    if minmax == 1:
        sol = -sol
        
    print(f"Function optimization value: {sol}")
    for i in range(len(vars)):
        print(f"{list(intvars.keys())[i]} = {round(intvars[list(intvars.keys())[i]].solution_value())}")
    
    solution_dictionary = {}
    for i in range(len(c)):
        solution_dictionary = {**solution_dictionary, f"{list(intvars.keys())[i]}": round(intvars[list(intvars.keys())[i]].solution_value())}

    return sol, solution_dictionary

c = [300,400]
A_ub = [[1,1],[120,200], [10,20]]
# A_eq = [[1,1]]
b_ub = [100,20000,1200]
# b_eq = [2000]

Optimized_cost, products = optimize(c,A_ub,b_ub,minmax=1)

['x0', 'x1']
<class 'ortools.linear_solver.pywraplp.Variable'>
Optimization completed
Function optimization value: 32000.0
x0 = 80
x1 = 20


In [334]:
print(Optimized_cost)
print(products)

32000.0
{'x0': 80, 'x1': 20}
